In [1]:
#import libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
%matplotlib inline

In [2]:
# Read datasets
df = pd.read_csv('air_quality.csv')
df.head()

C:\Users\mange\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,stn_code,sampling_date,state,location,agency,type,so2,no2,rspm,spm,location_monitoring_station,pm2_5,date
0,150.0,February - M021990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",4.8,17.4,NaN,NaN,NaN,NaN,1990-02-01
1,151.0,February - M021990,Andhra Pradesh,Hyderabad,NaN,Industrial Area,3.1,7.0,NaN,NaN,NaN,NaN,1990-02-01
2,152.0,February - M021990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",6.2,28.5,NaN,NaN,NaN,NaN,1990-02-01
3,150.0,March - M031990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",6.3,14.7,NaN,NaN,NaN,NaN,1990-03-01
4,151.0,March - M031990,Andhra Pradesh,Hyderabad,NaN,Industrial Area,4.7,7.5,NaN,NaN,NaN,NaN,1990-03-01


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435742 entries, 0 to 435741
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   stn_code                     291665 non-null  object 
 1   sampling_date                435739 non-null  object 
 2   state                        435742 non-null  object 
 3   location                     435739 non-null  object 
 4   agency                       286261 non-null  object 
 5   type                         430349 non-null  object 
 6   so2                          401096 non-null  float64
 7   no2                          419509 non-null  float64
 8   rspm                         395520 non-null  float64
 9   spm                          198355 non-null  float64
 10  location_monitoring_station  408251 non-null  object 
 11  pm2_5                        9314 non-null    float64
 12  date                         435735 non-null  object 
dtyp

In [4]:
df.shape

(435742, 13)

In [5]:
df.isna().sum()

stn_code                       144077
sampling_date                       3
state                               0
location                            3
agency                         149481
type                             5393
so2                             34646
no2                             16233
rspm                            40222
spm                            237387
location_monitoring_station     27491
pm2_5                          426428
date                                7
dtype: int64

In [6]:
# stn_code, agency, sampling_date, location_monitoring_agency do not add much value to the dataset 
# in terms of information. Therefore, we can drop those columns.

df=df.drop(['stn_code', 'agency','sampling_date','location_monitoring_station'],axis=1) 
df.head(2)

,state,location,type,so2,no2,rspm,spm,pm2_5,date
0,Andhra Pradesh,Hyderabad,"Residential, Rural and other Areas",4.8,17.4,NaN,NaN,NaN,1990-02-01
1,Andhra Pradesh,Hyderabad,Industrial Area,3.1,7.0,NaN,NaN,NaN,1990-02-01


In [7]:
# Dropping rows where no date is available.

df=df.dropna(subset=['date'])
df.shape

(435735, 9)

In [8]:
# defining columns of importance, which shall be used reguarly 
COLS = ['so2', 'no2', 'rspm', 'spm', 'pm2_5']
from sklearn.impute import SimpleImputer
# invoking SimpleImputer to fill missing values
imputer = SimpleImputer(missing_values=np.nan, strategy='mean') 
df[COLS] = imputer.fit_transform(df[COLS])
df.isna().sum()

state          0
location       0
type        5390
so2            0
no2            0
rspm           0
spm            0
pm2_5          0
date           0
dtype: int64

In [9]:
df['type'].value_counts()

Residential, Rural and other Areas    179013
Industrial Area                        96089
Residential and others                 86791
Industrial Areas                       51747
Sensitive Area                          8979
Sensitive Areas                         5536
RIRUO                                   1304
Sensitive                                495
Industrial                               233
Residential                              158
Name: type, dtype: int64

In [10]:
df['type'].replace({"Residential, Rural and other Areas":"RRO", "Industrial Area":"I", 
                    "Industrial Areas":"I","Industrial":"I", "Residential and others":"RO",
                    "Sensitive Area":"S","Sensitive Areas":"S","Sensitive":"S","RIRUO":"RIRUO",
                    "Residential":"R"},inplace= True) 
df['type'].value_counts()

RRO      179013
I        148069
RO        86791
S         15010
RIRUO      1304
R           158
Name: type, dtype: int64

In [11]:
df['type'].replace({"RRO":1, "I":2, "RO":3,"S":4,"RIRUO":5,"R":6},inplace= True) 
df['type']

0         1.0
1         2.0
2         1.0
3         1.0
4         2.0
         ... 
435734    5.0
435735    5.0
435736    5.0
435737    5.0
435738    5.0
Name: type, Length: 435735, dtype: float64

In [12]:
df['state'].value_counts()

Maharashtra             60382
Uttar Pradesh           42816
Andhra Pradesh          26368
Punjab                  25634
Rajasthan               25589
Kerala                  24728
Himachal Pradesh        22896
West Bengal             22463
Gujarat                 21279
Tamil Nadu              20597
Madhya Pradesh          19920
Assam                   19361
Odisha                  19278
Karnataka               17118
Delhi                    8551
Chandigarh               8520
Chhattisgarh             7831
Goa                      6206
Jharkhand                5968
Mizoram                  5338
Telangana                3978
Meghalaya                3853
Puducherry               3785
Haryana                  3420
Nagaland                 2463
Bihar                    2275
Uttarakhand              1961
Jammu & Kashmir          1289
Daman & Diu               782
Dadra & Nagar Haveli      634
Uttaranchal               285
Arunachal Pradesh          90
Manipur                    76
Sikkim    

In [13]:
from sklearn.preprocessing import LabelEncoder 
labelencoder=LabelEncoder() 
df["state"]=labelencoder.fit_transform(df["state"]) 
df.head(5)

,state,location,type,so2,no2,rspm,spm,pm2_5,date
0,0,Hyderabad,1.0,4.8,17.4,108.833091,220.78348,40.791467,1990-02-01
1,0,Hyderabad,2.0,3.1,7.0,108.833091,220.78348,40.791467,1990-02-01
2,0,Hyderabad,1.0,6.2,28.5,108.833091,220.78348,40.791467,1990-02-01
3,0,Hyderabad,1.0,6.3,14.7,108.833091,220.78348,40.791467,1990-03-01
4,0,Hyderabad,2.0,4.7,7.5,108.833091,220.78348,40.791467,1990-03-01


In [14]:
dfAndhra=df[(df['state']==0)] 
dfAndhra

,state,location,type,so2,no2,rspm,spm,pm2_5,date
0,0,Hyderabad,1.0,4.8,17.4,108.833091,220.78348,40.791467,1990-02-01
1,0,Hyderabad,2.0,3.1,7.0,108.833091,220.78348,40.791467,1990-02-01
2,0,Hyderabad,1.0,6.2,28.5,108.833091,220.78348,40.791467,1990-02-01
3,0,Hyderabad,1.0,6.3,14.7,108.833091,220.78348,40.791467,1990-03-01
4,0,Hyderabad,2.0,4.7,7.5,108.833091,220.78348,40.791467,1990-03-01
...,...,...,...,...,...,...,...,...,...
26363,0,Rajahmundry,2.0,7.0,13.0,71.000000,220.78348,40.791467,2015-12-13
26364,0,Rajahmundry,2.0,7.0,18.0,77.000000,220.78348,40.791467,2015-12-16
26365,0,Rajahmundry,2.0,8.0,23.0,64.000000,220.78348,40.791467,2015-12-19
26366,0,Rajahmundry,2.0,7.0,19.0,61.000000,220.78348,40.791467,2015-12-22


In [15]:
dfAndhra['location'].value_counts()

Hyderabad         7764
Visakhapatnam     7108
Vijayawada        2093
Chittoor          1003
Tirupati           986
Kurnool            857
Patancheru         698
Guntur             629
Nalgonda           618
Ramagundam         554
Nellore            408
Khammam            385
Warangal           336
Ananthapur         324
Ongole             317
Kadapa             316
Srikakulam         315
Rajahmundry        311
Eluru              300
Vishakhapatnam     297
Kakinada           288
Vizianagaram       282
Sangareddy          85
Karimnagar          67
Nizamabad           27
Name: location, dtype: int64

In [16]:
from sklearn.preprocessing import OneHotEncoder 
onehotencoder=OneHotEncoder(sparse=False,handle_unknown='error',drop='first')
pd.DataFrame(onehotencoder.fit_transform(dfAndhra[["location"]]))


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df['location'].value_counts()

Guwahati      9984
Hyderabad     9667
Delhi         8551
Chandigarh    8520
Jaipur        7850
              ... 
Bhuj            24
SANAND          23
Khajuraho       16
Dharuhera        1
Gangtok          1
Name: location, Length: 304, dtype: int64

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 435735 entries, 0 to 435738
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   state     435735 non-null  int32  
 1   location  435735 non-null  object 
 2   type      430345 non-null  float64
 3   so2       435735 non-null  float64
 4   no2       435735 non-null  float64
 5   rspm      435735 non-null  float64
 6   spm       435735 non-null  float64
 7   pm2_5     435735 non-null  float64
 8   date      435735 non-null  object 
dtypes: float64(6), int32(1), object(2)
memory usage: 31.6+ MB


In [21]:
df.head()

,state,location,type,so2,no2,rspm,spm,pm2_5,date
0,0,Hyderabad,1.0,4.8,17.4,108.833091,220.78348,40.791467,1990-02-01
1,0,Hyderabad,2.0,3.1,7.0,108.833091,220.78348,40.791467,1990-02-01
2,0,Hyderabad,1.0,6.2,28.5,108.833091,220.78348,40.791467,1990-02-01
3,0,Hyderabad,1.0,6.3,14.7,108.833091,220.78348,40.791467,1990-03-01
4,0,Hyderabad,2.0,4.7,7.5,108.833091,220.78348,40.791467,1990-03-01


In [22]:
df['date'].value_counts()

2015-03-19    253
2015-02-12    237
2015-02-19    236
2015-11-05    235
2015-11-11    234
             ... 
1988-10-08      1
2003-10-23      1
1988-04-27      1
1987-01-21      1
1987-07-13      1
Name: date, Length: 5067, dtype: int64

In [23]:
df['date'].nunique()

5067

In [29]:
df['year'] = pd.to_datetime(df['date'], format='%Y-%m-%d').dt.year
df['month'] = pd.to_datetime(df['date'], format='%Y-%m-%d').dt.month
df['day'] = pd.to_datetime(df['date'], format='%Y-%m-%d').dt.day
df.head()

,state,location,type,so2,no2,rspm,spm,pm2_5,date,year,month,day
0,0,Hyderabad,1.0,4.8,17.4,108.833091,220.78348,40.791467,1990-02-01,1990,2,1
1,0,Hyderabad,2.0,3.1,7.0,108.833091,220.78348,40.791467,1990-02-01,1990,2,1
2,0,Hyderabad,1.0,6.2,28.5,108.833091,220.78348,40.791467,1990-02-01,1990,2,1
3,0,Hyderabad,1.0,6.3,14.7,108.833091,220.78348,40.791467,1990-03-01,1990,3,1
4,0,Hyderabad,2.0,4.7,7.5,108.833091,220.78348,40.791467,1990-03-01,1990,3,1


In [30]:
df['year'].value_counts()

2015    50319
2013    45803
2014    44215
2011    37641
2012    35101
2010    34975
2007    34376
2008    32751
2006    28862
2009    28749
2005    19336
2004    16119
2003     2556
2002     2157
2001     2080
1996     2013
1997     1973
2000     1932
1999     1872
1995     1869
1998     1839
1991     1616
1994     1557
1990     1485
1993     1420
1992     1227
1989      823
1988      641
1987      428
Name: year, dtype: int64

In [31]:
df.drop(['date'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 435735 entries, 0 to 435738
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   state     435735 non-null  int32  
 1   location  435735 non-null  object 
 2   type      430345 non-null  float64
 3   so2       435735 non-null  float64
 4   no2       435735 non-null  float64
 5   rspm      435735 non-null  float64
 6   spm       435735 non-null  float64
 7   pm2_5     435735 non-null  float64
 8   year      435735 non-null  int64  
 9   month     435735 non-null  int64  
 10  day       435735 non-null  int64  
dtypes: float64(6), int32(1), int64(3), object(1)
memory usage: 38.2+ MB
